### 1. 테스트 함수 정의 (I/O vs CPU)
- I/O 바운드 작업: time.sleep()을 사용해서 네트워크 통신이나 디스크 읽기/쓰기처럼 CPU는 쉬고 '대기'하는 시간을 흉내 냅니다.
- CPU 바운드 작업: 복잡한 수학 연산(e.g., (i**2 + i**3) ** 0.5)을 수백만 번 반복시켜서 CPU를 혹사시킵니다.

In [1]:
import time
from dask import delayed


# 1. I/O-bound task (0.5초 대기)
def io_task(i):
    time.sleep(0.5)
    return i + 1


# 2. CPU-bound task (CPU를 100% 사용하며 연산)
def cpu_task(i):
    total = 0
    for j in range(10_000_000):  # 1천만 번 반복
        total += (j**2 + j**3) ** 0.5
    return total + i

### 2. 테스트 시나리오: 8개 작업을 병렬로
total_io와 total_cpu 모두 8개의 독립적인 작업이 완료된 후 sum으로 합쳐지는 그래프를 가지고 있습니다.\
이론적으로 8개의 작업을 병렬로 처리할 수 있다면, I/O 작업은 (8 * 0.5초 = 4초)가 아니라 약 0.5초 만에, CPU 작업도 (8 * (CPU 작업 시간))이 아니라 **(CPU 작업 시간)**에 가깝게 끝나야 합니다.

In [2]:
from dask import delayed

# I/O 작업 8개 리스트
io_tasks = [delayed(io_task)(i) for i in range(8)]
total_io = delayed(sum)(io_tasks)

# CPU 작업 8개 리스트
cpu_tasks = [delayed(cpu_task)(i) for i in range(8)]
total_cpu = delayed(sum)(cpu_tasks)

### 3. 스케줄러별 성능 테스트

In [3]:
%%time
# 테스트 1. scheduler='sync' (동기식 - 기준선)
print("--- 'sync' (I/O) 테스트 ---")
total_io.compute(scheduler='sync')

--- 'sync' (I/O) 테스트 ---
CPU times: user 758 ms, sys: 63.7 ms, total: 821 ms
Wall time: 7.36 s


36

In [4]:
%%time
print("--- 'sync' (CPU) 테스트 ---")
total_cpu.compute(scheduler='sync')

--- 'sync' (CPU) 테스트 ---
CPU times: user 27 s, sys: 0 ns, total: 27 s
Wall time: 24.5 s


1.0119288090904536e+18

In [5]:
%%time
# 테스트 2. scheduler='threads' (스레드 기반)
# Dask의 기본 스케줄러. I/O 바운드 작업에 최적화되어 있음.
print("--- 'threads' (I/O) 테스트 ---")
total_io.compute(scheduler='threads')

--- 'threads' (I/O) 테스트 ---
CPU times: user 12.1 ms, sys: 0 ns, total: 12.1 ms
Wall time: 507 ms


36

In [6]:
%%time
# sync 스케줄러와 거의 동일합니다. 병렬 처리의 이득이 전혀 없습니다.
print("--- 'threads' (CPU) 테스트 ---")
total_cpu.compute(scheduler='threads')

--- 'threads' (CPU) 테스트 ---
CPU times: user 28 s, sys: 512 ms, total: 28.5 s
Wall time: 28.9 s


1.0119288090904536e+18

In [7]:
%%time
# 테스트 3. scheduler="processes"
# 별도의 Python 프로세스들을 여러 개 띄웁니다.
print("--- 'processes' (CPU) 테스트 ---")
total_cpu.compute(scheduler='processes')

--- 'processes' (CPU) 테스트 ---
CPU times: user 8.23 ms, sys: 13.1 ms, total: 21.4 ms
Wall time: 22.4 s


1.0119288090904536e+18